In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

\begin{eqnarray}
\dot \delta &=& \Omega_b \left(\omega - \omega_s\right)\\
\dot \omega &=& 1/(2 H) \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{eqnarray}



## System definition 

In [2]:
params_dict = {'X_d':1.81,'X1d':0.3,'T1d0':8.0,
               'X_q':1.76,'X1q':0.65,'T1q0':1.0,
               'R_a':0.003,'X_l': 0.05, 
               'H':3.5,'D':1.0,
               'Omega_b':2*np.pi*50,'omega_s':1.0,
               'v_0':0.9008,'theta_0':0.0}


u_ini_dict = {'P_t':0.8, 'Q_t':0.2}  # for the initialization problem
u_run_dict = {'p_m':0.8,'e1q':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega']    # [inductor current, PI integrator]
y_ini_list = ['p_e','i_d','i_q','v_d','v_q','v_1','theta_1','p_m','e1q'] # for the initialization problem
y_run_list = ['p_e','i_d','i_q','v_d','v_q','v_1','theta_1','P_t','Q_t'] # for the running problem 

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
 

ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))

g_1 = (v_q + R_a*i_q)*i_q + (v_d + R_a*i_d)*i_d  - p_e 
g_2 = v_q + R_a*i_q - e1q + X1q*i_d 
g_3 = v_d + R_a*i_d - X1d*i_q
g_4 = v_1*sin(delta - theta_1) - v_d
g_5 = v_1*cos(delta - theta_1) - v_q
g_6 = i_d*v_d + i_q*v_q - P_t
g_7 = i_d*v_q - i_q*v_d - Q_t
g_8 = -P_t - (v_1*v_0*sin(theta_1 - theta_0))/X_l
g_9 = -Q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_l - v_1**2/X_l


f_list = [ddelta,domega]
g_list = [g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9]
h_dict = {'p_m':p_m,'p_e':p_e}

sys_dict = {'name':'smib_milano_ex8p1',
            'params_dict':params_dict,
            'f_list':f_list,
            'g_list':g_list,
            'x_list':x_list,
            'y_ini_list':y_ini_list,
            'y_run_list':y_run_list,
            'u_run_dict':u_run_dict,
            'u_ini_dict':u_ini_dict,
            'h_dict':h_dict}


db.system(sys_dict)
db.sys2num(sys_dict)
                

In [4]:
sys_dict['g']

Matrix([
[       i_d*(R_a*i_d + v_d) + i_q*(R_a*i_q + v_q) - p_e],
[                         R_a*i_q + X1q*i_d - e1q + v_q],
[                               R_a*i_d - X1d*i_q + v_d],
[                        v_1*sin(delta - theta_1) - v_d],
[                        v_1*cos(delta - theta_1) - v_q],
[                              -P_t + i_d*v_d + i_q*v_q],
[                              -Q_t + i_d*v_q - i_q*v_d],
[             -P_t + v_0*v_1*sin(theta_0 - theta_1)/X_l],
[-Q_t + v_0*v_1*cos(theta_0 - theta_1)/X_l - v_1**2/X_l]])